# 0.0 Divide your data into distributed train, val and test sets

Our file todo.txt contains all the images and its corresponding classes. Now we want to split our images into train, val and test files, so that the classes are well distributed (e.g. if the dataset is highly imbalanced the images are not randomly separated).

In [4]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd

# import imgclas
from imgclas import paths, config
# from imgclas.data_utils import load_image, load_data_splits, augment, load_class_names

from sklearn.model_selection import train_test_split

CONF = config.get_conf_dict()

# Customize your image folder if needed
# CONF['general']['images_directory'] = '/media/ignacio/Datos/datasets/semillas/datasets'  # absolute path to image_folder

# splits_dir = paths.get_splits_dir()
splits_dir = '/srv/Files_Envio_Definitivo/SPIDER/'

# Load the data
data = pd.read_csv(splits_dir+'Patients_SPIDER_all.txt', sep="*", header=None, names=['image', 'clase'])

In [5]:
data

,image,clase
0,/srv/datos/ENVIO_DEFINITIVO/NO_TORTUOSOS_OFICI...,0
1,/srv/datos/ENVIO_DEFINITIVO/NO_TORTUOSOS_OFICI...,0
2,/srv/datos/ENVIO_DEFINITIVO/NO_TORTUOSOS_OFICI...,0
3,/srv/datos/ENVIO_DEFINITIVO/NO_TORTUOSOS_OFICI...,0
4,/srv/datos/ENVIO_DEFINITIVO/NO_TORTUOSOS_OFICI...,0
...,...,...
396,/srv/datos/ENVIO_DEFINITIVO/TORTUOSOS_SPIDER/9...,1
397,/srv/datos/ENVIO_DEFINITIVO/TORTUOSOS_SPIDER/7...,1
398,/srv/datos/ENVIO_DEFINITIVO/TORTUOSOS_SPIDER/4...,1
399,/srv/datos/ENVIO_DEFINITIVO/TORTUOSOS_SPIDER/6...,1


In [6]:
data.clase.value_counts()

0    204
1    197
Name: clase, dtype: int64

In [54]:
# cross validation https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html
# Stratified K Fold Cross Validation
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)
X=data.image
y=data.clase
count = 1
# split()  method generate indices to split data into training and test set
for train_index, test_val_index in kf.split(X, y):
    X_train, X_test_val = X[train_index], X[test_index]
    y_train, y_test_val = y[train_index], y[test_index]
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=1, stratify=y_test_val)
    # train
    train=pd.DataFrame(data={'image': X_train, 'clase': y_train})
    train.to_csv(r'/srv/Files_Envio_Oficial/45/train'+str(count)+'.txt', sep=' ', index=None, header=None, mode='a')
    # val
    val=pd.DataFrame(data={'image': X_val, 'clase': y_val})
    val.to_csv(r'/srv/Files_Envio_Oficial/45/val'+str(count)+'.txt', sep=' ', index=None, header=None, mode='a')
    # test
    test=pd.DataFrame(data={'image': X_test, 'clase': y_test})
    test.to_csv(r'/srv/Files_Envio_Oficial/45/test'+str(count)+'.txt', sep=' ', index=None, header=None, mode='a')
    print(f'Fold:{count}, Train set: {len(train_index)}, Val set:{len(y_val)}, Test set:{len(y_test)}')
    count+=1

Fold:1, Train set: 320, Val set:40, Test set:40
Fold:2, Train set: 320, Val set:40, Test set:40
Fold:3, Train set: 320, Val set:40, Test set:40
Fold:4, Train set: 320, Val set:40, Test set:40
Fold:5, Train set: 320, Val set:40, Test set:40


In [3]:
# https://machisnelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/
from collections import Counter
print(Counter(data.clase))

Counter({0: 201, 1: 200})


In [5]:
# split into train test sets
X_train, X_2, y_train, y_2 = train_test_split(data.image, data.clase, test_size=0.15, random_state=1, stratify=data.clase)
print(Counter(y_train))
print(Counter(y_2))

Counter({1: 170, 0: 170})
Counter({0: 31, 1: 30})


In [8]:
X_test, X_val, y_test, y_val = train_test_split(X_2, y_2, test_size=0.49, random_state=1, stratify=y_2)
print(Counter(y_test))
print(Counter(y_val))

Counter({0: 16, 1: 15})
Counter({1: 15, 0: 15})


In [9]:
train=pd.DataFrame(data={'image': X_train, 'clase': y_train})
train.to_csv(r'/srv/image-classification-tf/data/dataset_files/train.txt', sep=' ', index=None, header=None, mode='a')

In [10]:
val=pd.DataFrame(data={'image': X_val, 'clase': y_val})
val.to_csv(r'/srv/image-classification-tf/data/dataset_files/val.txt', sep=' ', index=None, header=None, mode='a')

In [11]:
test=pd.DataFrame(data={'image': X_test, 'clase': y_test})
test.to_csv(r'/srv/image-classification-tf/data/dataset_files/test.txt', sep=' ', index=None, header=None, mode='a')

In [12]:
len(train)

340

In [13]:
len(test)

31

In [14]:
len(val)

30

In [65]:
# append spider and 45 angiographies together to create only 5 different models
# Reading data from file1
with open(r'/srv/Files_Envio_Oficial/45/test5.txt') as fp:
    data = fp.read()
    
with open(r'/srv/Files_Envio_Oficial/45/val5.txt') as fp:
    data3 = fp.read()
    
# Reading data from file2
with open(r'/srv/Files_Envio_Oficial/SPIDER/test5.txt') as fp:
    data2 = fp.read()
    
with open(r'/srv/Files_Envio_Oficial/SPIDER/val5.txt') as fp:
    data4 = fp.read()
    
# Merging 2 files
# To add the data of file2
# from next line
data += data2
data3 += data4
  
with open (r'/srv/Files_Envio_Oficial/test5', 'a') as fp:
    fp.write(data)
    
with open (r'/srv/Files_Envio_Oficial/val5', 'a') as fp:
    fp.write(data3)